In [270]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression,LinearRegression, SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm, grid_search
import pandas as pd
import numpy as np

In [271]:
ROOT_DIR = "../../"
dataset = pd.read_csv(ROOT_DIR+"data/structured/training_data_match_statistics.csv")
dataset = dataset.drop(dataset.columns[0],axis=1)

In [34]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression,LinearRegression, SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm, grid_search
import pandas as pd
import numpy as np

# 
# Creates MATCH statistics for all tournament matches in a season (for all seasons)
# 

ROOT_DIR = "../../"
dataset = pd.read_csv(ROOT_DIR+"data/structured/training_data_match_statistics.csv")
dataset = dataset.drop(dataset.columns[0],axis=1)

def getPredictionsFromClassifier(classifier,training_X,training_Y,testing_X,testing_Y):
    # Training    
    classifier.fit(training_X,np.ravel(training_Y))
    predicted_Y = classifier.predict(testing_X)
    predicted_probabilities_Y = classifier.predict_proba(testing_X)[:,0]

#     print(classification_report(predicted_Y.round().astype(int),testing_Y))
#     print(accuracy_score(predicted_Y.round().astype(int),testing_Y))

    return predicted_Y,predicted_probabilities_Y

def classNameForClassifier(classifier):
    class_name = str(classifier.__class__)
    class_name = class_name[class_name.rfind(".")+1:class_name.rfind("'>")]
    return class_name

def addPredictionAndClassifierToList(prediction_probabilities_for_winning,classifier,predicted_probabilities_Y):
    class_name = classNameForClassifier(classifier)
    prediction = pd.DataFrame(predicted_probabilities_Y,columns=[class_name])
    prediction_probabilities_for_winning = pd.concat([prediction_probabilities_for_winning,prediction],axis=1)
    return prediction_probabilities_for_winning

# Dataset Partitioning
dataset = pd.read_csv(ROOT_DIR+"data/structured/training_data_match_statistics.csv")
dataset = dataset.drop(dataset.columns[0],axis=1)

training_data = dataset[(dataset.season != 2014) & (dataset.season > 2014-4)]
testing_data = dataset[dataset.season == 2014]

#Preprocess/Filter Data here
training_X = training_data[training_data.columns[1:len(training_data.columns)-1]].fillna(0)
training_Y = training_data[training_data.columns[-1]]

testing_X = testing_data[testing_data.columns[1:len(testing_data.columns)-1]].fillna(0)
testing_Y = testing_data[testing_data.columns[-1]]

testing_X = training_X
testing_Y = training_Y

classifiers = [RandomForestClassifier(),LogisticRegression()]
prediction_probabilities_for_winning = pd.DataFrame()

for classifier in classifiers:
    predicted_Y, predicted_probabilities_Y = getPredictionsFromClassifier(classifier,training_X,training_Y,testing_X,testing_Y)
    prediction_probabilities_for_winning = addPredictionAndClassifierToList(prediction_probabilities_for_winning,classifier,predicted_probabilities_Y)   
    
training_Y = pd.DataFrame(training_Y.as_matrix(),columns=["winningTeam"])

ensemble_training_X = prediction_probabilities_for_winning
ensemble_training_Y = training_Y

testing_X = testing_data[testing_data.columns[1:len(testing_data.columns)-1]].fillna(0)
testing_Y = testing_data[testing_data.columns[-1]]

prediction_probabilities_for_winning = pd.DataFrame()

for classifier in classifiers:
    predicted_Y, predicted_probabilities_Y = getPredictionsFromClassifier(classifier,training_X,training_Y,testing_X,testing_Y)
    prediction_probabilities_for_winning = addPredictionAndClassifierToList(prediction_probabilities_for_winning,classifier,predicted_probabilities_Y)   
    
ensemble_testing_X = prediction_probabilities_for_winning
ensemble_testing_Y = testing_Y    

ensemble_classifier = LogisticRegression()
ensemble_classifier.fit(ensemble_training_X,np.ravel(ensemble_training_Y))

y_pred = ensemble_classifier.predict(ensemble_testing_X)
print(classification_report(y_pred,ensemble_testing_Y))
print(accuracy_score(y_pred,ensemble_testing_Y))

y_pred_prob = ensemble_classifier.predict_proba(ensemble_testing_X)[:,0]

             precision    recall  f1-score   support

          0       0.61      0.72      0.66        32
          1       0.69      0.57      0.62        35

avg / total       0.65      0.64      0.64        67

0.641791044776


In [36]:
y_pred_prob.shape

(67,)

In [39]:
testing_data.columns

Index([u'season', u'wscore', u'wfgm', u'wfga', u'wfgm3', u'wfga3', u'wftm', u'wfta', u'wor', u'wdr', u'wast', u'wto', u'wstl', u'wblk', u'wpf', u'lscore', u'lfgm', u'lfga', u'lfgm3', u'lfga3', u'lftm', u'lfta', u'lor', u'ldr', u'last', u'lto', u'lstl', u'lblk', u'lpf', u'winningTeam'], dtype='object')

In [55]:
# Dataset Partitioning
dataset = pd.read_csv(ROOT_DIR+"data/structured/training_data_match_statistics.csv")
dataset = dataset.drop(dataset.columns[0],axis=1)
submission = pd.read_csv(ROOT_DIR+"data/structured/prediction_probabilities_for_matchups.csv")

target_season = 2014
testing_data = dataset[dataset.season == target_season]

#Preprocess/Filter Data here
startingFeatureIndex = 3

testing_Y = testing_data[testing_data.columns[-1]].reset_index().drop("index",axis=1).as_matrix()
submission_Y = submission[submission.columns[-1]].reset_index().drop("index",axis=1).as_matrix()
0.5 * np.ones(testing_Y.shape[0])

In [143]:
testing_Y = testing_data[testing_data.columns[-1]].reset_index().drop("index",axis=1).as_matrix()
submission_Y = submission[submission.columns[-1]].reset_index().drop("index",axis=1).as_matrix()
bench_Y = 0.5 * np.ones(testing_Y.shape[0])

In [145]:
y = testing_Y
log_y_hat = np.log(submission_Y)
_1_y = 1 - testing_Y
_log_1_y_hat = np.log(1 - submission_Y)
logLoss = (-0.9/y.shape[0]) * (np.dot(y.transpose(),log_y_hat) + np.dot(_1_y.transpose(),_log_1_y_hat))
logLoss

array([[ 0.95929326]])

In [131]:
submission_Y = bench_Y
log_y_hat = np.log(submission_Y)
_log_1_y_hat = np.log(1 - submission_Y)
logLoss = (-1.0/y.shape[0]) * (np.dot(y.transpose(),log_y_hat) + np.dot(_1_y.transpose(),_log_1_y_hat))
logLoss

array([ 0.69314718])

In [102]:
np.dot(y,log_y_hat)

array([[-26.77149069]])